In [1]:
!pip install gdown==4.6.0

  Attempting uninstall: gdown
    Found existing installation: gdown 4.7.3
    Uninstalling gdown-4.7.3:
      Successfully uninstalled gdown-4.7.3


In [2]:
! gdown 1Ktw_3F5vP7OJxS6bLO4kxJNY5BxR0Rhp

Downloading...
From: https://drive.google.com/uc?id=1Ktw_3F5vP7OJxS6bLO4kxJNY5BxR0Rhp
To: /content/final_df.csv
100% 1.79M/1.79M [00:00<00:00, 106MB/s]


In [28]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv('/content/final_df.csv')

In [5]:
df

,Lot Frontage,Lot Area,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Mas Vnr Area,BsmtFin SF 1,BsmtFin SF 2,Bsmt Unf SF,...,Sale Type_ConLw,Sale Type_New,Sale Type_Oth,Sale Type_VWD,Sale Type_WD,Sale Condition_AdjLand,Sale Condition_Alloca,Sale Condition_Family,Sale Condition_Normal,Sale Condition_Partial
0,141.000000,31770,6,5,1960,1960,112.0,639.0,0.0,441.0,...,0,0,0,0,1,0,0,0,1,0
1,80.000000,11622,5,6,1961,1961,0.0,468.0,144.0,270.0,...,0,0,0,0,1,0,0,0,1,0
2,81.000000,14267,6,6,1958,1958,108.0,923.0,0.0,406.0,...,0,0,0,0,1,0,0,0,1,0
3,93.000000,11160,7,5,1968,1968,0.0,1065.0,0.0,1045.0,...,0,0,0,0,1,0,0,0,1,0
4,74.000000,13830,5,5,1997,1998,0.0,791.0,0.0,137.0,...,0,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2920,37.000000,7937,6,6,1984,1984,0.0,819.0,0.0,184.0,...,0,0,0,0,1,0,0,0,1,0
2921,75.144444,8885,5,5,1983,1983,0.0,301.0,324.0,239.0,...,0,0,0,0,1,0,0,0,1,0
2922,62.000000,10441,5,5,1992,1992,0.0,337.0,0.0,575.0,...,0,0,0,0,1,0,0,0,1,0
2923,77.000000,10010,5,5,1974,1975,0.0,1071.0,123.0,195.0,...,0,0,0,0,1,0,0,0,1,0


In [6]:
X = df.drop('SalePrice', axis =1)

In [7]:
y = df['SalePrice']

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state = 101)

In [10]:
from sklearn.preprocessing import StandardScaler

In [11]:
scaler = StandardScaler()

In [12]:
scaled_X_train = scaler.fit_transform(X_train)


In [13]:
scaled_X_test = scaler.transform(X_test)

In [14]:
from sklearn.linear_model import ElasticNet

In [20]:
elastic_model = ElasticNet(max_iter = 1000000)

In [21]:
param_grid = {'alpha':[0.1, 1, 5, 10, 100], 'l1_ratio':[0.1, 0.7, 0.99, 1]}

In [17]:
from sklearn.model_selection import GridSearchCV

In [22]:
grid_model = GridSearchCV(elastic_model, param_grid = param_grid, scoring = 'neg_mean_squared_error', cv = 5, verbose = 1)

In [23]:
grid_model.fit(scaled_X_train, y_train)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


GridSearchCV(cv=5, estimator=ElasticNet(max_iter=1000000),
             param_grid={'alpha': [0.1, 1, 5, 10, 100],
                         'l1_ratio': [0.1, 0.7, 0.99, 1]},
             scoring='neg_mean_squared_error', verbose=1)

In [24]:

 grid_model.best_params_

{'alpha': 100, 'l1_ratio': 1}

In [25]:
y_pred = grid_model.predict(scaled_X_test)

In [26]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [27]:
mean_absolute_error(y_test, y_pred)

14195.35490056217

In [29]:
np.sqrt(mean_squared_error(y_test, y_pred))

20558.508566893164

In [30]:
np.mean(df['SalePrice'])

180815.53743589742

**Погрешность модели - около 10%.**

Далее попробуем использовать Catboost. Т.к. в этом случае dummy-переменные не требуются, то работать будем не с файлом final_df.csv, а с файлом df_cleared.

In [31]:
! gdown 1nyKvsUfhqxnlY8nElzLZcEG2-_aRtXBy

Downloading...
From: https://drive.google.com/uc?id=1nyKvsUfhqxnlY8nElzLZcEG2-_aRtXBy
To: /content/df_cleared.csv
100% 950k/950k [00:00<00:00, 41.7MB/s]


In [32]:
df1 = pd.read_csv('/content/df_cleared.csv')

In [33]:
df1['MS SubClass']  = df1['MS SubClass'].apply(str)

In [35]:
train, test = train_test_split(df1, test_size = 0.4, random_state =101)

In [36]:
val, test = train_test_split(test, test_size = 0.5, random_state =101)

In [37]:
train.columns

Index(['MS SubClass', 'MS Zoning', 'Lot Frontage', 'Lot Area', 'Street',
       'Lot Shape', 'Land Contour', 'Utilities', 'Lot Config', 'Land Slope',
       'Neighborhood', 'Condition 1', 'Condition 2', 'Bldg Type',
       'House Style', 'Overall Qual', 'Overall Cond', 'Year Built',
       'Year Remod/Add', 'Roof Style', 'Roof Matl', 'Exterior 1st',
       'Exterior 2nd', 'Mas Vnr Type', 'Mas Vnr Area', 'Exter Qual',
       'Exter Cond', 'Foundation', 'Bsmt Qual', 'Bsmt Cond', 'Bsmt Exposure',
       'BsmtFin Type 1', 'BsmtFin SF 1', 'BsmtFin Type 2', 'BsmtFin SF 2',
       'Bsmt Unf SF', 'Total Bsmt SF', 'Heating', 'Heating QC', 'Central Air',
       'Electrical', '1st Flr SF', '2nd Flr SF', 'Low Qual Fin SF',
       'Gr Liv Area', 'Bsmt Full Bath', 'Bsmt Half Bath', 'Full Bath',
       'Half Bath', 'Bedroom AbvGr', 'Kitchen AbvGr', 'Kitchen Qual',
       'TotRms AbvGrd', 'Functional', 'Fireplaces', 'Fireplace Qu',
       'Garage Type', 'Garage Yr Blt', 'Garage Finish', 'Garage Cars',

In [39]:
X = ['MS SubClass', 'MS Zoning', 'Lot Frontage', 'Lot Area', 'Street',
       'Lot Shape', 'Land Contour', 'Utilities', 'Lot Config', 'Land Slope',
       'Neighborhood', 'Condition 1', 'Condition 2', 'Bldg Type',
       'House Style', 'Overall Qual', 'Overall Cond', 'Year Built',
       'Year Remod/Add', 'Roof Style', 'Roof Matl', 'Exterior 1st',
       'Exterior 2nd', 'Mas Vnr Type', 'Mas Vnr Area', 'Exter Qual',
       'Exter Cond', 'Foundation', 'Bsmt Qual', 'Bsmt Cond', 'Bsmt Exposure',
       'BsmtFin Type 1', 'BsmtFin SF 1', 'BsmtFin Type 2', 'BsmtFin SF 2',
       'Bsmt Unf SF', 'Total Bsmt SF', 'Heating', 'Heating QC', 'Central Air',
       'Electrical', '1st Flr SF', '2nd Flr SF', 'Low Qual Fin SF',
       'Gr Liv Area', 'Bsmt Full Bath', 'Bsmt Half Bath', 'Full Bath',
       'Half Bath', 'Bedroom AbvGr', 'Kitchen AbvGr', 'Kitchen Qual',
       'TotRms AbvGrd', 'Functional', 'Fireplaces', 'Fireplace Qu',
       'Garage Type', 'Garage Yr Blt', 'Garage Finish', 'Garage Cars',
       'Garage Area', 'Garage Qual', 'Garage Cond', 'Paved Drive',
       'Wood Deck SF', 'Open Porch SF', 'Enclosed Porch', '3Ssn Porch',
       'Screen Porch', 'Pool Area', 'Misc Val', 'Mo Sold', 'Yr Sold',
       'Sale Type', 'Sale Condition']

In [40]:
y = ['SalePrice']

In [43]:
df1.select_dtypes(include = 'object').columns

Index(['MS SubClass', 'MS Zoning', 'Street', 'Lot Shape', 'Land Contour',
       'Utilities', 'Lot Config', 'Land Slope', 'Neighborhood', 'Condition 1',
       'Condition 2', 'Bldg Type', 'House Style', 'Roof Style', 'Roof Matl',
       'Exterior 1st', 'Exterior 2nd', 'Mas Vnr Type', 'Exter Qual',
       'Exter Cond', 'Foundation', 'Bsmt Qual', 'Bsmt Cond', 'Bsmt Exposure',
       'BsmtFin Type 1', 'BsmtFin Type 2', 'Heating', 'Heating QC',
       'Central Air', 'Electrical', 'Kitchen Qual', 'Functional',
       'Fireplace Qu', 'Garage Type', 'Garage Finish', 'Garage Qual',
       'Garage Cond', 'Paved Drive', 'Sale Type', 'Sale Condition'],
      dtype='object')

In [45]:
cat_features = ['MS SubClass', 'MS Zoning', 'Street', 'Lot Shape', 'Land Contour',
       'Utilities', 'Lot Config', 'Land Slope', 'Neighborhood', 'Condition 1',
       'Condition 2', 'Bldg Type', 'House Style', 'Roof Style', 'Roof Matl',
       'Exterior 1st', 'Exterior 2nd', 'Mas Vnr Type', 'Exter Qual',
       'Exter Cond', 'Foundation', 'Bsmt Qual', 'Bsmt Cond', 'Bsmt Exposure',
       'BsmtFin Type 1', 'BsmtFin Type 2', 'Heating', 'Heating QC',
       'Central Air', 'Electrical', 'Kitchen Qual', 'Functional',
       'Fireplace Qu', 'Garage Type', 'Garage Finish', 'Garage Qual',
       'Garage Cond', 'Paved Drive', 'Sale Type', 'Sale Condition']

In [47]:
! pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 2.9 MB/s eta 0:00:00


In [48]:
from catboost import CatBoostRegressor

In [49]:
cb_model = CatBoostRegressor(cat_features = cat_features, eval_metric = 'MAE')

In [50]:
cb_model.fit(train[X], train[y], eval_set = (val[X], val[y]))

Learning rate set to 0.055622
0:	learn: 56052.0103378	test: 57927.9945428	best: 57927.9945428 (0)	total: 130ms	remaining: 2m 10s
1:	learn: 53788.5852874	test: 55721.0859010	best: 55721.0859010 (1)	total: 214ms	remaining: 1m 46s
2:	learn: 51599.9764954	test: 53464.1547822	best: 53464.1547822 (2)	total: 281ms	remaining: 1m 33s
3:	learn: 49550.2806500	test: 51551.1355760	best: 51551.1355760 (3)	total: 345ms	remaining: 1m 25s
4:	learn: 47530.5017083	test: 49541.0804044	best: 49541.0804044 (4)	total: 410ms	remaining: 1m 21s
5:	learn: 45747.8592585	test: 47818.3504493	best: 47818.3504493 (5)	total: 477ms	remaining: 1m 19s
6:	learn: 43913.2693219	test: 45970.5098537	best: 45970.5098537 (6)	total: 541ms	remaining: 1m 16s
7:	learn: 42430.4247876	test: 44512.6797956	best: 44512.6797956 (7)	total: 582ms	remaining: 1m 12s
8:	learn: 40970.5586534	test: 43089.8176395	best: 43089.8176395 (8)	total: 651ms	remaining: 1m 11s
9:	learn: 39602.2492591	test: 41786.6070910	best: 41786.6070910 (9)	total: 713m

985 - точка переобучения.

In [52]:
cb_model.predict(test[X])

array([134721.78270874, 170223.5238006 , 251449.80094746, 137444.50469418,
       262926.48608272,  94817.66027092, 321579.61904192, 389690.985929  ,
       186562.58955393, 187671.95770723, 166836.42105264, 141360.54506284,
       126395.51183584, 199963.65746917, 345198.46795041, 102578.72821657,
       147367.44543743, 197536.66830315, 367281.86335492, 126337.42215243,
       399163.63659193, 127713.69615901, 141756.64599454, 155061.61047296,
       268859.33834258, 139283.6837859 , 178043.24108648, 169456.90255541,
       156112.09986177,  96939.1007967 , 192004.21802239, 148185.03977352,
       137309.45185389, 152902.3098584 , 194268.92621945,  67907.40619161,
       221581.18092639, 107588.27912958, 141791.54272466, 155279.82887359,
       322360.53924495, 142367.48519436, 135112.26786298, 286629.81151198,
       195404.6880793 , 355902.42646417, 227872.74329637,  85628.79354261,
       178866.81279717, 138786.8041711 , 188189.22969517, 130475.60060018,
       127407.73437133, 1

In [53]:
test['price_pred'] = cb_model.predict(test[X])

In [54]:
test

,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Lot Shape,Land Contour,Utilities,Lot Config,Land Slope,...,3Ssn Porch,Screen Porch,Pool Area,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice,price_pred
1448,20,RL,71.336364,9216,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,9,2008,WD,Abnorml,143195,134721.782709
1145,20,RL,70.669811,7153,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,6,2008,WD,Normal,179200,170223.523801
1459,60,RL,71.336364,9240,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,189,0,0,9,2008,WD,Normal,287000,251449.800947
1872,20,RL,70.000000,8050,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,3,2007,WD,Normal,127000,137444.504694
544,20,RL,88.000000,10994,Pave,IR1,Lvl,AllPub,Corner,Gtl,...,0,0,0,0,9,2009,COD,Abnorml,257000,262926.486083
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252,20,RL,64.000000,7360,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,3,2010,WD,Normal,187000,191368.702431
989,20,RL,87.000000,11029,Pave,IR1,Lvl,AllPub,Corner,Gtl,...,0,190,0,0,5,2008,WD,Normal,176500,179537.833231
403,160,RM,24.000000,2368,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,5,2009,WD,Normal,125000,122103.590489
1443,60,RL,68.000000,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,9,2008,WD,Normal,223500,209551.000175


In [55]:
mean_absolute_error(test['SalePrice'], test['price_pred'])

13136.37829614983

In [56]:
np.sqrt(mean_squared_error(test['SalePrice'], test['price_pred']))

21107.08149912713

In [62]:
train_full = pd.concat([train, val])

In [58]:
cb_model.best_iteration_

985

In [60]:
cb_model_full = CatBoostRegressor(cat_features = cat_features, eval_metric = 'MAE', iterations = cb_model.best_iteration_ + 1)

In [63]:
cb_model_full.fit(train_full[X], train_full[y])


Learning rate set to 0.047369
0:	learn: 56878.7390561	total: 70ms	remaining: 1m 8s
1:	learn: 54872.7801712	total: 138ms	remaining: 1m 7s
2:	learn: 52994.6902698	total: 211ms	remaining: 1m 9s
3:	learn: 51054.9741951	total: 265ms	remaining: 1m 5s
4:	learn: 49307.9355459	total: 331ms	remaining: 1m 4s
5:	learn: 47560.8970980	total: 383ms	remaining: 1m 2s
6:	learn: 46002.8633737	total: 446ms	remaining: 1m 2s
7:	learn: 44378.0225931	total: 515ms	remaining: 1m 2s
8:	learn: 43056.1166469	total: 576ms	remaining: 1m 2s
9:	learn: 41661.2697435	total: 639ms	remaining: 1m 2s
10:	learn: 40309.5247659	total: 699ms	remaining: 1m 1s
11:	learn: 39004.6095907	total: 755ms	remaining: 1m 1s
12:	learn: 37774.8858966	total: 804ms	remaining: 1m
13:	learn: 36528.0053401	total: 852ms	remaining: 59.2s
14:	learn: 35387.0847248	total: 920ms	remaining: 59.5s
15:	learn: 34253.7624060	total: 995ms	remaining: 1m
16:	learn: 33268.7154952	total: 1.06s	remaining: 1m
17:	learn: 32307.7098060	total: 1.14s	remaining: 1m 1s


In [64]:
cb_model_full.predict(test[X])

array([127497.95405465, 176347.24536555, 259949.81538095, 141792.20634412,
       253721.54788231,  95827.77554485, 300234.71376145, 402681.92898088,
       190054.00635071, 184275.575076  , 172891.44960768, 137441.49782222,
       122501.53058977, 197612.41688657, 325307.95902232,  99676.13811803,
       151051.65571644, 210308.41039841, 362184.19317043, 126822.89338175,
       400291.31910605, 126794.22133015, 139634.82677774, 157608.79343844,
       268561.40420653, 139193.19432738, 181453.25427634, 176266.50836527,
       154019.5547402 , 102713.71182707, 196163.00813045, 146606.430497  ,
       133850.54950713, 154792.37902073, 191711.03512643,  70450.31647309,
       223311.7741665 , 106948.29814532, 141984.61566497, 155546.46357317,
       329753.68667314, 137981.22195028, 132728.74836676, 299882.13847153,
       198922.22170871, 349391.68552082, 220895.17604722,  92103.16261382,
       175496.92178742, 142832.1031739 , 187366.3694398 , 126885.98917715,
       125978.22120147, 2

In [65]:
test['price_pred_full'] = cb_model_full.predict(test[X])

In [66]:
mean_absolute_error(test['SalePrice'], test['price_pred_full'])

12578.432677790179

In [67]:
np.sqrt(mean_squared_error(test['SalePrice'], test['price_pred_full']))

19792.768170655814

In [68]:
from joblib import dump

In [69]:
dump(cb_model, 'cb_model.joblib')

['cb_model.joblib']